In [1]:
# CONTEXTO

# Vamos a ver si podemos mejorar los resultados del modelo LGBM único
# Para ello haremos un stacking, pero no en paralelo, sino en segmentado
# Haremos 3 subgrupos de hoteles. Grandes, pequeños y medianos
# Para ello haremos un value_counts de todo el dataset, para evitar comernos algún hotel que esté en test y no en train o viceversa
# Aunque sea una mala praxis tocar el test para todo aquello que no sea evaluación, en este caso simplemente haremos una separación y si un modelo está en test pero no en train o viceversa acabaría correspondiéndole desde un punto de vista conceptual el grupo de los pequeños.
# Así que en esta ocasión, podemos tomarnos la licencia.
# En cuanto al resto de los hoteles, medianos y grandes, deberían tener un share parecido en train y test... Así que todo ok

#### Preparativos

In [3]:
# Importaciones

import pandas as pd
import numpy as np

In [4]:
# Deslimitar/Limitar display Pandas

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_colwidth', None)

#### Preparación Dataset

In [5]:
# Read Matches data

df0 = pd.read_csv(r'D:\DEV\Python\00_TFM_PALLADIUM\02_DATASETS_GENERADOS\Reservas_Feature_Engineered_v1.csv', sep = ';', decimal=',')

C:\Users\Diego\AppData\Local\Temp\ipykernel_852\2151874057.py:3: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.
  df0 = pd.read_csv(r'D:\DEV\Python\00_TFM_PALLADIUM\02_DATASETS_GENERADOS\Reservas_Feature_Engineered_v1.csv', sep = ';', decimal=',')


In [6]:
df0.head()

,ID_RESERVA,ID_HOTEL,HOTEL,LLEGADA,LLEGADA_ANO,LLEGADA_MES,LLEGADA_DIAm,LLEGADA_DIAs,LLEGADA_AVANCE,SALIDA,SALIDA_ANO,SALIDA_MES,SALIDA_DIAm,SALIDA_DIAs,SALIDA_AVANCE,NOCHES,DURACION_ESTANCIA,REGIMEN,ID_TIPO,TIPO,USO,PAX_NUM,PAX_CAT,ADULTOS,NENES,BEBES,ID_CLIENTE,TIPO_CLIENTE,CLIENTE,GRUPO,ID_MONEDA,MONEDA,STATUS,MOTIVO,CHECKIN,SUPLETORIA,CUNAS,FECHA_TOMA,FECHA_TOMA_ANO,FECHA_TOMA_MES,FECHA_TOMA_DIAm,FECHA_TOMA_DIAs,FECHA_TOMA_AVANCE,FECHA_MOD,FECHA_MOD_ANO,FECHA_MOD_MES,FECHA_MOD_DIAm,FECHA_MOD_DIAs,FECHA_MOD_AVANCE,FECHA_CANCELACION,FECHA_CANCELACION_ANO,FECHA_CANCELACION_MES,FECHA_CANCELACION_DIAm,FECHA_CANCELACION_DIAs,FECHA_CANCELACION_AVANCE,LT_TOMA_LLEGADA,LT_TOMA_CANCELACION,ID_FIDELIDAD,FIDELIDAD,VALHAB,VALPEN,VALSERV,VALFIJOS,COMERCIALIZADORA,CMVALHAB,CMVALPEN,CMCVALSERV,VALOR_USD,VALOR_USD_PAX,VALOR_USD_NOCHE,VALOR_USD_PAX_NOCHE,AUTORIZO,GRATIS,PAIS,CONTINENTE,SEGMENTO,FUENTE_NEGOCIO,CANCELADA
0,456094009201,92,GrandPalladiumJamaica&LadyHamiltonResort,2022-12-28,2022,12,28,3,0.989041095890411,2023-01-04,2023,1,4,3,0.00821917808219178,7,Media,AllInclusive,23300,LHAmbassadorSuiteBSOV,2,3,Familias,2,1,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,1,DESCONOCIDO_0,1,0,0,2022-01-13,2022,1,13,4,0.03287671232876712,2022-12-28,2022,12.0,28.0,3.0,0.989041095890411,NaN,NaT,NaN,NaN,NaN,NaN,349,NaN,0,Ninguna,1868.0,708.0,0.0,0.0,0,1868.0,708.0,0.0,2576.0,858.6666666666666,368.0,122.66666666666666,1,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,0
1,425835909201,92,GrandPalladiumJamaica&LadyHamiltonResort,2022-01-22,2022,1,22,6,0.057534246575342465,2022-01-29,2022,1,29,6,0.07671232876712329,7,Media,AllInclusive,43800,LHJuniorSuiteGV,2,3,Familias,2,1,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2021-07-13,2021,7,13,2,0.5287671232876713,2022-01-20,2022,1.0,20.0,4.0,0.052054794520547946,2022-01-20,2022,1.0,20.0,4.0,0.052054794520547946,193,191.0,0,Ninguna,1329.3,665.0,0.0,0.0,0,1329.3,665.0,0.0,1994.3,664.7666666666667,284.9,94.96666666666667,1,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,1
2,549929510601,106,ComplejoRivieraMaya,2023-11-06,2023,11,6,1,0.8465753424657534,2023-11-14,2023,11,14,2,0.8684931506849315,8,Media,AllInclusive,42000,TRSJuniorSuitePS,2,2,Parejas,2,0,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2023-04-17,2023,4,17,1,0.29041095890410956,2023-11-07,2023,11.0,7.0,2.0,0.8493150684931506,2023-10-17,2023,10.0,17.0,2.0,0.7917808219178082,203,183.0,0,Ninguna,1146.56,1146.56,0.0,12.48,0,1146.56,1146.56,0.0,2305.6,1152.8,288.2,144.1,1,0,CANADA,AMERICA,Fixedrates,DIRECTSALES,1
3,519229110601,106,ComplejoRivieraMaya,2023-11-06,2023,11,6,1,0.8465753424657534,2023-11-23,2023,11,23,4,0.8931506849315068,17,Larga,AllInclusive,42000,TRSJuniorSuitePS,2,2,Parejas,2,0,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2023-02-22,2023,2,22,3,0.14246575342465753,2023-11-07,2023,11.0,7.0,2.0,0.8493150684931506,2023-09-14,2023,9.0,14.0,4.0,0.7013698630136986,257,204.0,0,Ninguna,2436.44,2436.44,0.0,26.52,0,2436.44,2436.44,0.0,4899.400000000001,2449.7000000000003,288.20000000000005,144.10000000000002,1,0,ESTADOSUNIDOS,AMERICA,Fixedrates,DIRECTSALES,1
4,537405910601,106,ComplejoRivieraMaya,2023-11-25,2023,11,25,6,0.8986301369863013,2023-12-09,2023,12,9,6,0.936986301369863,14,Larga,AllInclusive,13000,COLJuniorSuiteGV,1,1,Single,1,0,0,19686200,1,PALLADIUMTRAVELCLUB_SOCIOS,0,700,USD,3,DESCONOCIDO_0,0,0,0,2023-02-17,2023,2,17,5,0.12876712328767123,2023-11-26,2023,11.0,26.0,7.0,0.9013698630136986,2023-04-11,2023,4.0,11.0,2.0,0.273972602739726,281,53.0,0,Ninguna,2413.18,0.0,0.0,21.84,0,2413.18,0.0,0.0,2435.02,2435.02,173.93,173.93,1,0,RUMANIA,EURESTE,Fixedrates,DIRECTSALES,1


In [7]:
df0.columns

Index(['ID_RESERVA', 'ID_HOTEL', 'HOTEL', 'LLEGADA', 'LLEGADA_ANO',
       'LLEGADA_MES', 'LLEGADA_DIAm', 'LLEGADA_DIAs', 'LLEGADA_AVANCE',
       'SALIDA', 'SALIDA_ANO', 'SALIDA_MES', 'SALIDA_DIAm', 'SALIDA_DIAs',
       'SALIDA_AVANCE', 'NOCHES', 'DURACION_ESTANCIA', 'REGIMEN', 'ID_TIPO',
       'TIPO', 'USO', 'PAX_NUM', 'PAX_CAT', 'ADULTOS', 'NENES', 'BEBES',
       'ID_CLIENTE', 'TIPO_CLIENTE', 'CLIENTE', 'GRUPO', 'ID_MONEDA', 'MONEDA',
       'STATUS', 'MOTIVO', 'CHECKIN', 'SUPLETORIA', 'CUNAS', 'FECHA_TOMA',
       'FECHA_TOMA_ANO', 'FECHA_TOMA_MES', 'FECHA_TOMA_DIAm',
       'FECHA_TOMA_DIAs', 'FECHA_TOMA_AVANCE', 'FECHA_MOD', 'FECHA_MOD_ANO',
       'FECHA_MOD_MES', 'FECHA_MOD_DIAm', 'FECHA_MOD_DIAs', 'FECHA_MOD_AVANCE',
       'FECHA_CANCELACION', 'FECHA_CANCELACION_ANO', 'FECHA_CANCELACION_MES',
       'FECHA_CANCELACION_DIAm', 'FECHA_CANCELACION_DIAs',
       'FECHA_CANCELACION_AVANCE', 'LT_TOMA_LLEGADA', 'LT_TOMA_CANCELACION',
       'ID_FIDELIDAD', 'FIDELIDAD', 'VALHAB', '

#### Preparación de un DF Mixto

In [8]:
df_mix = df0.copy()

In [9]:
# Reconfigurar dtypes

df_mix.drop(columns=['ID_RESERVA'], inplace=True) # No usar, ID
df_mix.drop(columns=['ID_HOTEL'], inplace=True) # No usar, ID
df_mix['HOTEL'] = df_mix['HOTEL'].astype('category', errors='raise')
df_mix.drop(columns=['LLEGADA'], inplace=True) # No usar, contamina extrapolación
df_mix.drop(columns=['LLEGADA_ANO'], inplace=True) # No usar, contamina extrapolación
df_mix['LLEGADA_MES'] = pd.to_numeric(df_mix['LLEGADA_MES'], errors='raise').astype('category')
df_mix['LLEGADA_DIAm'] = pd.to_numeric(df_mix['LLEGADA_DIAm'], errors='raise').astype('category')
df_mix['LLEGADA_DIAs'] = pd.to_numeric(df_mix['LLEGADA_DIAs'], errors='raise').astype('category')
df_mix['LLEGADA_AVANCE'] = pd.to_numeric(df_mix['LLEGADA_AVANCE'], errors='raise').astype(float)
df_mix.drop(columns=['SALIDA'], inplace=True) # No usar, contamina extrapolación
df_mix.drop(columns=['SALIDA_ANO'], inplace=True) # No usar, contamina extrapolación
df_mix['SALIDA_MES'] = pd.to_numeric(df_mix['SALIDA_MES'], errors='raise').astype('category')
df_mix['SALIDA_DIAm'] = pd.to_numeric(df_mix['SALIDA_DIAm'], errors='raise').astype('category')
df_mix['SALIDA_DIAs'] = pd.to_numeric(df_mix['SALIDA_DIAs'], errors='raise').astype('category')
df_mix['SALIDA_AVANCE'] = pd.to_numeric(df_mix['SALIDA_AVANCE'], errors='raise').astype(float)
df_mix['NOCHES'] = pd.to_numeric(df_mix['NOCHES'], errors='raise').astype('Int64')
df_mix['DURACION_ESTANCIA'] = df_mix['DURACION_ESTANCIA'].astype('category', errors='raise')
df_mix['REGIMEN'] = df_mix['REGIMEN'].astype('category', errors='raise')
df_mix.drop(columns=['ID_TIPO'], inplace=True) # No usar, ID
df_mix['TIPO'] = df_mix['TIPO'].astype('category', errors='raise')
df_mix['USO'] = pd.to_numeric(df_mix['USO'], errors='raise').astype('Int64')
df_mix['PAX_NUM'] = pd.to_numeric(df_mix['PAX_NUM'], errors='raise').astype('Int64')
df_mix['PAX_CAT'] = df_mix['PAX_CAT'].astype('category', errors='raise')
df_mix['ADULTOS'] = pd.to_numeric(df_mix['ADULTOS'], errors='raise').astype('Int64')
df_mix['NENES'] = pd.to_numeric(df_mix['NENES'], errors='raise').astype('Int64')
df_mix['BEBES'] = pd.to_numeric(df_mix['BEBES'], errors='raise').astype('Int64')
df_mix.drop(columns=['ID_CLIENTE'], inplace=True) # No usar, ID
df_mix['TIPO_CLIENTE'] = pd.to_numeric(df_mix['TIPO_CLIENTE'], errors='raise').astype('category')
df_mix['CLIENTE'] = df_mix['CLIENTE'].astype('category', errors='raise')
df_mix['GRUPO'] = pd.to_numeric(df_mix['GRUPO'], errors='raise').astype('category')
df_mix.drop(columns=['ID_MONEDA'], inplace=True) # No usar, ID
df_mix['MONEDA'] = df_mix['MONEDA'].astype('category', errors='raise')
df_mix.drop(columns=['STATUS'], inplace=True) # No usar, redundante con variable objetivo
df_mix.drop(columns=['MOTIVO'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['CHECKIN'], inplace=True) # No usar, imposible ver el futuro
df_mix['SUPLETORIA'] = pd.to_numeric(df_mix['SUPLETORIA'], errors='raise').astype('Int64')
df_mix['CUNAS'] = pd.to_numeric(df_mix['CUNAS'], errors='raise').astype('Int64')
df_mix.drop(columns=['FECHA_TOMA'], inplace=True) # No usar, contamina extrapolación
df_mix.drop(columns=['FECHA_TOMA_ANO'], inplace=True) # No usar, contamina extrapolación
df_mix['FECHA_TOMA_MES'] = pd.to_numeric(df_mix['FECHA_TOMA_MES'], errors='raise').astype('category')
df_mix['FECHA_TOMA_DIAm'] = pd.to_numeric(df_mix['FECHA_TOMA_DIAm'], errors='raise').astype('category')
df_mix['FECHA_TOMA_DIAs'] = pd.to_numeric(df_mix['FECHA_TOMA_DIAs'], errors='raise').astype('category')
df_mix['FECHA_TOMA_AVANCE'] = pd.to_numeric(df_mix['FECHA_TOMA_AVANCE'], errors='raise').astype(float)
df_mix.drop(columns=['FECHA_MOD'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_ANO'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_MES'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_DIAm'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_DIAs'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_MOD_AVANCE'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_ANO'], inplace=True) # No usar, contamina extrapolación, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_MES'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_DIAm'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_DIAs'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['FECHA_CANCELACION_AVANCE'], inplace=True) # No usar, imposible ver el futuro
df_mix['LT_TOMA_LLEGADA'] = pd.to_numeric(df_mix['LT_TOMA_LLEGADA'], errors='raise').astype('Int64')
df_mix.drop(columns=['LT_TOMA_CANCELACION'], inplace=True) # No usar, imposible ver el futuro
df_mix.drop(columns=['ID_FIDELIDAD'], inplace=True) # No usar, ID
df_mix['FIDELIDAD'] = df_mix['FIDELIDAD'].astype('category', errors='raise')
df_mix.drop(columns=['VALHAB'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['VALPEN'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['VALSERV'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['VALFIJOS'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix['COMERCIALIZADORA'] = pd.to_numeric(df_mix['COMERCIALIZADORA'], errors='raise').astype('category')
df_mix.drop(columns=['CMVALHAB'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['CMVALPEN'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix.drop(columns=['CMCVALSERV'], inplace=True) # No usar, no está convertido y teniendo el valor total es redundante y/o dependiente de COMERCIALIZADORA
df_mix['VALOR_USD'] = pd.to_numeric(df_mix['VALOR_USD'], errors='raise').astype(float)
df_mix['VALOR_USD_PAX'] = pd.to_numeric(df_mix['VALOR_USD_PAX'], errors='raise').astype(float)
df_mix['VALOR_USD_NOCHE'] = pd.to_numeric(df_mix['VALOR_USD_NOCHE'], errors='raise').astype(float)
df_mix['VALOR_USD_PAX_NOCHE'] = pd.to_numeric(df_mix['VALOR_USD_PAX_NOCHE'], errors='raise').astype(float)
df_mix.drop(columns=['AUTORIZO'], inplace=True) # No usar, de momento no muy claro
df_mix['GRATIS'] = pd.to_numeric(df_mix['GRATIS'], errors='raise').astype('category')
df_mix['PAIS'] = df_mix['PAIS'].astype('category', errors='raise')
df_mix['CONTINENTE'] = df_mix['CONTINENTE'].astype('category', errors='raise')
df_mix['SEGMENTO'] = df_mix['SEGMENTO'].astype('category', errors='raise')
df_mix['FUENTE_NEGOCIO'] = df_mix['FUENTE_NEGOCIO'].astype('category', errors='raise')
df_mix['CANCELADA'] = pd.to_numeric(df_mix['CANCELADA'], errors='raise').astype('category')

In [10]:
df_mix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1139289 entries, 0 to 1139288
Data columns (total 42 columns):
 #   Column               Non-Null Count    Dtype   
---  ------               --------------    -----   
 0   HOTEL                1139289 non-null  category
 1   LLEGADA_MES          1139289 non-null  category
 2   LLEGADA_DIAm         1139289 non-null  category
 3   LLEGADA_DIAs         1139289 non-null  category
 4   LLEGADA_AVANCE       1139289 non-null  float64 
 5   SALIDA_MES           1139289 non-null  category
 6   SALIDA_DIAm          1139289 non-null  category
 7   SALIDA_DIAs          1139289 non-null  category
 8   SALIDA_AVANCE        1139289 non-null  float64 
 9   NOCHES               1139289 non-null  Int64   
 10  DURACION_ESTANCIA    1139289 non-null  category
 11  REGIMEN              1139289 non-null  category
 12  TIPO                 1139289 non-null  category
 13  USO                  1139289 non-null  Int64   
 14  PAX_NUM              1139289 non-n

#### Segmentación por tamaño del hotel (número de reservas)

In [11]:
df_mix['HOTEL'].value_counts(dropna=False)

HOTEL
ComplejoPuntaCana                           262616
ComplejoCostaMujeres                        211825
ComplejoRivieraMaya                         200133
GrandPalladiumJamaica&LadyHamiltonResort    175235
GrandPalladiumImbassaiResort&Spa            111886
PalladiumVallarta                           106676
DominicanFiestaHotel&Casino                  44237
TRSCapCana                                   26681
Name: count, dtype: int64

In [12]:
# Generamos la columna TAMANO_HOTEL inicializada a Grande
df_mix['TAMANO_HOTEL'] = 'Grande'
# Con el panorama que hemos visto en el value counts, categorizamos los hoteles así:
df_mix.loc[df_mix['HOTEL'] == 'GrandPalladiumJamaica&LadyHamiltonResort', 'TAMANO_HOTEL'] = 'Mediano'
df_mix.loc[df_mix['HOTEL'] == 'GrandPalladiumImbassaiResort&Spa', 'TAMANO_HOTEL'] = 'Mediano'
df_mix.loc[df_mix['HOTEL'] == 'PalladiumVallarta', 'TAMANO_HOTEL'] = 'Mediano'
df_mix.loc[df_mix['HOTEL'] == 'DominicanFiestaHotel&Casino', 'TAMANO_HOTEL'] = 'Pequeno'
df_mix.loc[df_mix['HOTEL'] == 'TRSCapCana', 'TAMANO_HOTEL'] = 'Pequeno'

In [13]:
df_mix['TAMANO_HOTEL'].value_counts()

TAMANO_HOTEL
Grande     674574
Mediano    393797
Pequeno     70918
Name: count, dtype: int64

In [15]:
# Exportamos para posterior tratamiento en otro Notebook
df_mix.to_csv(r'D:\DEV\Python\00_TFM_PALLADIUM\02_DATASETS_GENERADOS\Reservas_Stacking.csv', index=False, sep=';')